In [97]:
from datetime import datetime
import ctypes

In [2]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        print(f"__get__ called, self={self}, instance={instance}, owner_class={owner_class}")
        return datetime.utcnow().isoformat()

#### calling from class

In [3]:
class Logger1:
    current_time = TimeUTC()

class Logger2:
    current_time = TimeUTC()

In [4]:
Logger1.current_time

__get__ called, self=<__main__.TimeUTC object at 0x000001724D672988>, instance=None, owner_class=<class '__main__.Logger1'>


'2021-04-26T10:28:32.550620'

In [6]:
getattr(Logger1, "current_time")

__get__ called, self=<__main__.TimeUTC object at 0x000001724D672988>, instance=None, owner_class=<class '__main__.Logger1'>


'2021-04-26T10:29:19.951444'

In [7]:
Logger2.current_time

__get__ called, self=<__main__.TimeUTC object at 0x000001724D672EC8>, instance=None, owner_class=<class '__main__.Logger2'>


'2021-04-26T10:30:13.833071'

#### calling from instance

In [10]:
l1 = Logger1()
hex(id(l1))

'0x1724d8f4dc8'

In [11]:
l1.current_time

__get__ called, self=<__main__.TimeUTC object at 0x000001724D672988>, instance=<__main__.Logger1 object at 0x000001724D8F4DC8>, owner_class=<class '__main__.Logger1'>


'2021-04-26T10:31:39.128298'

In [13]:
l1_1 = Logger1()
l1_1.current_time

__get__ called, self=<__main__.TimeUTC object at 0x000001724D672988>, instance=<__main__.Logger1 object at 0x000001724D90BB88>, owner_class=<class '__main__.Logger1'>


'2021-04-26T10:32:51.747860'

#### so we can differentiate inside of __get__ method whether the descriptor was accessed via the class or via an instance of the class

In [14]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return datetime.utcnow().isoformat()

In [15]:
class Logger:
    current_time = TimeUTC()

In [16]:
Logger.current_time

In [17]:
Logger.__dict__

mappingproxy({'__module__': '__main__',
              'current_time': <__main__.TimeUTC at 0x1724d924c08>,
              '__dict__': <attribute '__dict__' of 'Logger' objects>,
              '__weakref__': <attribute '__weakref__' of 'Logger' objects>,
              '__doc__': None})

In [18]:
l = Logger()
l.current_time

'2021-04-26T10:44:31.809647'

this is consistant with the way properties work

In [20]:
class Logger:
    @property
    def current_time(self):
        return datetime.utcnow().isoformat()

In [21]:
Logger.current_time

In [22]:
l = Logger()
l.current_time

'2021-04-26T10:46:06.580579'

#### same instance of TimeUTC is called every time

In [33]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        print(f"__get__ called, self={self}")
        return datetime.utcnow().isoformat()
    
class Logger:
    current_time = TimeUTC()

In [34]:
l1 = Logger()
l2 = Logger()

In [35]:
l1.current_time, l2.current_time

__get__ called, self=<__main__.TimeUTC object at 0x000001724D84EDC8>
__get__ called, self=<__main__.TimeUTC object at 0x000001724D84EDC8>


('2021-04-26T10:57:16.186865', '2021-04-26T10:57:16.186865')

In [36]:
class Countdown:
    def __init__(self, start):
        self.start = start + 1
    def __get__(self, instance, owner_class):
        if instance is None: #don't use 'if not instance'
            return self
        self.start -= 1
        return self.start

In [37]:
class Rocket:
    countdown = Countdown(10)
rocket1 = Rocket()
rocket2 = Rocket()

In [38]:
rocket1.countdown

10

In [39]:
rocket2.countdown

9

In [40]:
rocket1.countdown

8

#### Setter

In [41]:
class IntegerValue:
    def __set__(self, instance, value):
        print(f"__set__ called, instance={instance}, value={value}")
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            print("__get__ called from class")
        else:
            print(f"__get__ called, instance={instance}, owner_class={owner_class}")


In [42]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [43]:
Point2D.x

__get__ called from class


In [44]:
p = Point2D()
p.x

__get__ called, instance=<__main__.Point2D object at 0x000001724D92CDC8>, owner_class=<class '__main__.Point2D'>


In [45]:
p.x = 100

__set__ called, instance=<__main__.Point2D object at 0x000001724D92CDC8>, value=100


In [46]:
p.x

__get__ called, instance=<__main__.Point2D object at 0x000001724D92CDC8>, owner_class=<class '__main__.Point2D'>


# where to store values for x and y

#### we do not want to store the values in descriptors directly. it works bad with more than one instance.

Bad example:

In [47]:
class IntegerValue:
    def __set__(self, instance, value):
        self._value = value
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            return self._value # will generate an exception if called before set


In [48]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [49]:
p1 = Point2D()
p1.x = 1.1
p1.y = 2.2

In [50]:
p1.x, p1.y

(1.1, 2.2)

In [51]:
p2 = Point2D()
p2.x = 10.1

In [52]:
p2.x

10.1

In [53]:
p1.x

10.1

### using instance dictionary for the storage

In [55]:
class IntegerValue:
    def __set__(self, instance, value):
        instance.stored_value = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            return getattr(instance, "stored_value", None)

In [56]:
class Point1D:
    x = IntegerValue()


In [57]:
p1, p2 = Point1D(), Point1D()

In [58]:
p1.x = 10.1
p2.x = 20.2

In [59]:
p1.x, p2.x

(10, 20)

In [60]:
p1.__dict__, p2.__dict__

({'stored_value': 10}, {'stored_value': 20})

our descriptor is hardcoded to use the same key in the instance dict. what if we use two instance descriptors in our class?

In [61]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [63]:
p = Point2D()
p.x = 10.1
p.y = 20.2

In [64]:
p.__dict__

{'stored_value': 20}

that's an issue

In [65]:
p.x, p.y

(20, 20)

### we need distinct name for each stored property

In [70]:
class IntegerValue:
    def __init__(self, name):
        self.storage_name = "_" + name
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            return getattr(instance, self.storage_name, None)

In [71]:
class Point2D:
    x = IntegerValue("x")
    y = IntegerValue("y")

In [72]:
p1, p2 = Point2D(), Point2D()

In [73]:
p1.x = 10.1
p1.y = 20.2

In [78]:
p1.__dict__

{'_x': 10.1, '_y': 20.2}

In [75]:
p2.__dict__

{}

In [76]:
p2.x = 100.1
p2.y = 200.2

In [77]:
p2.__dict__

{'_x': 100.1, '_y': 200.2}

 drawbacks:
 - we need to specify name of the property twice (x = IntegerValue("x"))
 - we are assuming that `_x` isn't already in use in the class 
 - not gonna work with slots

In [80]:
p1 = Point2D()
p1._x = 100
p1.__dict__

{'_x': 100}

In [81]:
p1.x = 200
p1.__dict__ #overwritten

{'_x': 200}

#### Let's assume that class is using slots and hashable
we will create storage in instance of descriptor

In [84]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None: 
            return self
        else:
            return self.values.get(instance)

In [85]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [86]:
p1, p2 = Point2D(), Point2D()

In [87]:
p1.x = 10.1
p1.y = 20.2
p1.x, p1.y

(10, 20)

In [88]:
Point2D.x.values

{<__main__.Point2D at 0x1724d8b8288>: 10}

In [89]:
p2.x = 100
Point2D.x.values

{<__main__.Point2D at 0x1724d8b8288>: 10,
 <__main__.Point2D at 0x1724d8b8948>: 100}

In [90]:
Point2D.y.values

{<__main__.Point2D at 0x1724d8b8288>: 20}

In [91]:
p2.y = 100
Point2D.y.values

{<__main__.Point2D at 0x1724d8b8288>: 20,
 <__main__.Point2D at 0x1724d8b8948>: 100}

#### this has fixed all the problems, but now we have memory leak

In [92]:
hex(id(p1))

'0x1724d8b8288'

In [93]:
del p1

In [94]:
Point2D.x.values # we still have a reference to '0x1724d8b8288' object

{<__main__.Point2D at 0x1724d8b8288>: 10,
 <__main__.Point2D at 0x1724d8b8948>: 100}

In [95]:
# we still can get it
p1 = list(Point2D.x.values.keys())[0]
p1

In [96]:
p1.x, p1.y

(10, 20)

In [98]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [99]:
p1 = Point2D()
id_p1 = id(p1)

In [100]:
ref_count(id_p1)

1

In [101]:
p1.x = 33
ref_count(id_p1)

2

In [103]:
"p1" in globals()

True

In [104]:
del p1

In [105]:
"p1" in globals()

False

In [106]:
ref_count(id_p1)

1